# Zonal statistics using polygons as zones and raster input for calculating statistics
---
Purpose: preprocess raster datasets for aggregation by SimU 

This code takes the following data inputs: 

1. national level SimUs as vector file
2. categorical* raster(s) for the years of interest

/* note: it will also accept ordinal rasters too, but you will need to then specify the stats to report

And produces a csv in the following format:

row 1: Name_1  |  FID_SimU_a  |  SimUID  |  HRU  |  Area_1000ha |  Class 1  | Class 2  |  Class n...  |  LULC_areaSum

Area of each class and area in LULC_areaSum are in units of 1000 ha

## Preamble

In [235]:
import gdal
import geopandas as gpd
import pandas as pd
import fiona as fi
import matplotlib
import matplotlib.pyplot as plt
from osgeo import ogr
import json
import geojson
import os
from rasterstats import zonal_stats
import rasterio
import numpy as np
import time

## 1) Function to calculate zonal stats

In [258]:
def calcZS(zones, rasterPath, statsList, categorical, zones_dropCols, convUnit):
    ## rasterstats manual: https://pythonhosted.org/rasterstats/manual.html#zonal-statistics

    ## calc zonal stats
    stats = zonal_stats(zones, rasterPath, categorical = categorical, stats=statsList, nodata = 0)

    ## convert stats output to pd df
    stats_df = pd.DataFrame(stats)
    
    ## change the index to be the same as zones to enable concatenation
    stats_df = stats_df.set_index(zones.index)
    
    ## convert pixel count to unit of interest
    stats_df = stats_df*convUnit
    ## Create new column that is the sum of all the LULC area categories
    stats_df["LULC_areaSum"] = stats_df.sum(axis = 1, skipna = True)
    
    ## concat the zones columns 
    stats_df_concat = pd.concat([pd.DataFrame(zones), stats_df], axis = 1, ignore_index = False)
    
    ## remove any excess columns from the zones df
    stats_df_concat.drop(columns = zones_dropCols, inplace = True)

    return stats_df_concat

## 2) Parameters 
set parameters, import datasets, set output file names

In [289]:
#### GENERAL PARAMETERS
dataType = "LULC_l48" ## Used to name file
year = 2016 ## used to name file
## columns in include in the final reclassified csv
columnsInFinal = ["UniqueID", "COUNTRY", "FID_countr", "HRU", "Grd30", "State", "FID_SimU_a", "SimUID", "Area_km2" ,"Area_1000ha", "LULC_areaSum", \
                  "Forest" , "CrpLnd", "Grass", "OthNatLnd", "WetLnd", "NotRel"]
projectVector = "yes" ## yes or not to project the zones/vector file to match the raster projection
testRun = "no" ## yes or no to run for just a state
testRun_state = "" ## if yes, this will be the state it will run
units = "1000ha" ## used to name file
viewMaxRows = 20

#### ZONAL STATS FUNCTION ARGUMENTS
## list: zones/vector column names to drop, if any
dropColsLs = ["geometry", 'oilp_yield', 'oilp_yie_1', 'oilp_yie_2']
## scalar: area of each grid cell to multiply the categorical count to get to units of 1000 ha
convUnit_a = 0.00009 
## for the above, convUnit_a = 0.00009 for landsat (30 m data) since equal to 900 m2 or 30m x 30m in units of 1000 ha (0.09 ha = 900 m2; divide by 10,000)
## boolean: whether or not the raster is categorical
categoricalRaster = True
## list: stats to calculate (none if categorical raster)
statList = []

#### INPUT FILE PATHS 
## Polygon (SimU) datast
SimU_fp = "C:\\Users\\Grace\\Documents\\FABLE\\GLOBIOM\\SimU\\gis\\SimU_all_select840_USstateSplit_proj.shp"

## Raster dataset
raster_fp= "C:\\Users\\Grace\\Documents\\FABLE\\GLOBIOM\\US_data\\LULC\\NLCD\\NLCD_2016_Land_Cover_L48_20190424\\NLCD_2016_Land_Cover_L48_20190424.img"

## legend
legend_fp = "C:\\Users\\Grace\\Documents\\FABLE\\GLOBIOM\\US_data\\LULC\\NLCD\\legend.csv"
leg_col_1 = "Code"
leg_col_2 = "Name"

## recclassification legend
reclassLegend_fp = "C:\\Users\\Grace\\Documents\\FABLE\\GLOBIOM\\US_data\\LULC\\NLCD\\NCLD_GLOBIOM_mapping.csv"
reclass_col_1 = "NLCD_categoryName"
reclass_col_2 = "GLOBIOM_CategoryName"

#### OUTPUT FILE PATHS
## csv filename of state land area totals (for checking later, not an output)
totalLandArea_fp = "C:\\Users\\Grace\\Documents\\FABLE\\GLOBIOM\\US_data\\LULC\\NLCD\\processed\\" + dataType + "_totalLandArea_" + units + "_" + str(year) + testRun_state + ".csv"

## csv filename of original classes
results_fp = "C:\\Users\\Grace\\Documents\\FABLE\\GLOBIOM\\US_data\\LULC\\NLCD\\processed\\" + dataType + "_NCLD_SimU_originalClasses_" + units + "_" + str(year) + testRun_state + ".csv"

## csv filename of reclassifed classes
results_mapped_fp = "C:\\Users\\Grace\\Documents\\FABLE\\GLOBIOM\\US_data\\LULC\\NLCD\\processed\\" + dataType + "_NCLD_SimU_GLOBIOMclasses_" + units + "_" + str(year) + testRun_state + ".csv"

## 3) Preprocess inputs as necessary

### If specified, project the vector/zones file to the same projection as the raster file

In [290]:
if projectVector == "yes":
    SimU = gpd.read_file(SimU_fp)
    raster = rasterio.open(raster_fp)

    SimU_proj = SimU.to_crs(crs=raster.crs.data)
    
else:
    SimU_proj = SimU

### Rename and calculate additional columns 

In [291]:
## add the units to the area column
SimU_proj.rename(columns={"Area": "Area_km2"}, inplace = True)

## convert units into 1000ha
SimU_proj["Area_1000ha"] = SimU_proj["Area_km2"]*0.1

### Test run: only use zones from a specified area

In [292]:
if testRun == "yes":
    print("Running " + testRun_state)
    SimU_proj = SimU_proj.loc[(SimU_proj['NAME_1'].astype(str) == testRun_state)]
    SimU_proj

## 4) Run zonal stats

In [ ]:
start_time = time.time()
## Run function
zs_out = calcZS(zones = SimU_proj, rasterPath = raster_fp, statsList = statList, categorical = categoricalRaster, zones_dropCols = dropColsLs, convUnit = convUnit_a)

## report time
elapsed_time = (time.time() - start_time)/(60)
print("^ Total time for completion: " + str(elapsed_time) + " minutes")

### View outputs

In [ ]:
with pd.option_context('display.max_rows', viewMaxRows, 'display.max_columns', None):  # more options can be specified also
    print(zs_out)

### Calculate total area within states for each state
Save to csv to compare with official state land area

In [ ]:
## total area in state in 1000ha or 0.1*km2
totLndArea = zs_out.groupby("NAME_1").LULC_areaSum.sum()
totLndArea.to_csv(totalLandArea_fp)

## 5) Replace numerical column names with the official class names and save as csv

In [ ]:
legend = pd.read_csv(legend_fp)

## convert the first two columns into a dictionary
legend_dict = dict(zip(legend[leg_col_1], legend[leg_col_2]))
#NLCDlegend_dict

## replace columns using dictionary
zs_out_names = zs_out.rename(columns=legend_dict)
zs_out_names

## rename columns as necessary
zs_out_names.rename(columns = {"NAME_1": "State"}, inplace = True)

### Export csv with original classes

In [ ]:
zs_out_names.to_csv(results_fp)

## 6) Reclassify for model requirements and save as another csv

This section can be run without running any of the above analysis, just fill out the parameters chunk

In [ ]:
## read output from source
zs_out_names = pd.read_csv(results_fp)
zs_out_names.rename(columns = {"Unnamed: 0": "UniqueID"}, inplace = True)

## read csv with mapping from NLCD to GLOBIOM
reclassLegend = pd.read_csv(reclassLegend_fp)

## convert the first two columns into a dictionary
reclassLegend_dict = dict(zip(reclassLegend[reclass_col_1], reclassLegend[reclass_col_2]))
#NLCDlegend_dict

## replace columns using dictionary
zs_out_names_GLO = zs_out_names.rename(columns=reclassLegend_dict)

## Sum the columns with the same names
zs_out_names_GLO_summed = zs_out_names_GLO.groupby(zs_out_names_GLO.columns, axis=1).sum()
zs_out_names_GLO_summed

## reorder columns
zs_out_names_GLO_summed = zs_out_names_GLO_summed[columnsInFinal]

## save to csv
zs_out_names_GLO_summed.to_csv(results_mapped_fp)